In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.linalg
import time

In [6]:
def construct_D(m, r):
    D_tail = np.ones(m-r) * 4e-3
    D_main = r - np.arange(1, r+1) + 1
    return np.diag(np.concatenate((D_main, D_tail)))

def generate_A(m, n, r):
    X = scipy.linalg.orth(np.random.randn(m, m))
    Y = scipy.linalg.orth(np.random.randn(n, m))
    D = construct_D(m, r)
    return X @ D @ Y.T, X, Y, D

def randomized_svd(A, left_c, right_c):
    A_fro_sq = np.power(np.linalg.norm(A, ord='fro'), 2)
    p_i = np.power(np.linalg.norm(A, axis=0), 2) / A_fro_sq
    col_inds = np.where(np.random.multinomial(left_c, p_i) > 0)[0]
    A_prime_left = A[:, col_inds] / np.sqrt(left_c * p_i[np.newaxis, col_inds])
    U, Sigma_left, _ = np.linalg.svd(A_prime_left, full_matrices=False)
    
    p_j = np.power(np.linalg.norm(A, axis=1), 2) / A_fro_sq
    row_inds = np.where(np.random.multinomial(right_c, p_j) > 0)[0]
    A_prime_right = A[row_inds, :] / np.sqrt(right_c * p_j[row_inds, np.newaxis])
    _, Sigma_right, Vt = np.linalg.svd(A_prime_right, full_matrices=False)
    
    return U, Vt.T, Sigma_left, Sigma_right

def compute_error_U(U_hat, U_true):
    def multiply_and_compute_norm(U_hat, U_true, x):
      prod = U_hat @ (U_hat.T @ x) - U_true @ (U_true.T @ x)
      prod = U_hat @ (U_hat.T @ prod) - U_true @ (U_true.T @ prod)
      norm = x.T @ prod 
      prod /= np.linalg.norm(prod)
      return prod, norm
    
    vec = np.random.randn(U_hat.shape[0], 1)
    vec /= np.linalg.norm(vec)
    prev_vec = np.zeros(vec.shape)
    while np.linalg.norm(vec - prev_vec) > 1e-6:
      prev_vec = vec
      vec, norm = multiply_and_compute_norm(U_hat, U_true, vec)
    return vec, norm

def compute_error_V(V_hat, V_true):
    def multiply_and_compute_norm(V_hat, V_true, x):
      prod = V_hat @ (V_hat.T @ x) - V_true @ (V_true.T @ x)
      prod = V_hat @ (V_hat.T @ prod) - V_true @ (V_true.T @ prod)
      norm = x.T @ prod 
      prod /= np.linalg.norm(prod)
      return prod, norm
    
    vec = np.random.randn(V_hat.shape[0], 1)
    vec /= np.linalg.norm(vec)
    prev_vec = np.zeros(vec.shape)
    while np.linalg.norm(vec - prev_vec) > 1e-6:
      prev_vec = vec
      vec, norm = multiply_and_compute_norm(V_hat, V_true, vec)
    return vec, norm


def search_C(A, U_true, V_true, r, eps, step_size=5):
  c_U, c_V, Ueps, Veps = 100, 50, np.inf, np.inf
  Ur_true, Vr_true = U_true[:, :r], V_true[:, :r]
  while abs(Ueps - eps) >= eps * 0.5 or abs(Veps - eps) >= eps * 0.5:
    U, V, _, _ = randomized_svd(A, c_U, c_V)
    Ur_hat, Vr_hat = U[:, :r], V[:, :r]
    if abs(Ueps - eps) >= eps * 0.5:
      _, Ueps = compute_error_U(Ur_hat, Ur_true)
      if Ueps > eps:
        c_U += step_size
      elif eps - Ueps >= eps * 0.5:
        c_U -= step_size
      
    if abs(Veps - eps) >= eps * 0.5:
      _, Veps = compute_error_V(Vr_hat, Vr_true)
      if Veps > eps:
        c_V += step_size
      elif eps - Veps >= eps * 0.5:
        c_V -= step_size
  
  print(f'Final: ueps {Ueps}, veps {Veps}')
  return c_U, c_V

### Q2.i

In [3]:
## Constants
m = 1000
n = 100000
r = 10
num_experiments = 10

In [4]:
A, X, Y, D = generate_A(m, n, r)
A_svd_start = time.time()
U_true, S_true, Vt_true = np.linalg.svd(A, full_matrices=False)
A_svd_end = time.time()
V_true = Vt_true.T

A_svd_runtime = A_svd_end - A_svd_start

In [7]:
results = {}
Ur_true, Vr_true = U_true[: , :r], V_true[:, :r]
for eps in [0.01, 0.05, 0.1]:
  this_res = {'true_params': {'r': r, 'A': A, 'U_true': U_true, 'V_true': V_true}}
  
  c_U, c_V = search_C(A, U_true, V_true, r, eps, step_size=5)
  print(f'For eps {eps}, U columns {c_U} and V columns {c_V}')
  this_res['c_U'], this_res['c_V'] = c_U, c_V
  
  run_time_list, U_err_list, V_err_list, approxA_params_list = [], [], [], []
  for i in range(num_experiments):
    random_svd_start = time.time()
    U, V, Sl, Sr = randomized_svd(A, c_U, c_V)
    random_svd_finish = time.time()
    
    U_err_list.append(compute_error_U(U[:, :r], Ur_true)[1])
    V_err_list.append(compute_error_V(V[:, :r], Vr_true)[1])
      
    approxA_params_list.append({'U': U, 'V': V, 'Sl': Sl, 'Sr': Sr})
    run_time_list.append(random_svd_finish - random_svd_start)

  this_res['ave_run_time'] = np.mean(run_time_list)
  this_res['ave_U_err'] = np.mean(U_err_list)
  this_res['ave_V_err'] = np.mean(V_err_list)
  
  results[eps] = this_res

Final: ueps [[0.01374737]], veps [[0.00856539]]
For eps 0.01, U columns 15 and V columns 15
Final: ueps [[0.07443312]], veps [[0.02771036]]
For eps 0.05, U columns 15 and V columns 10
Final: ueps [[0.05043966]], veps [[0.13454916]]
For eps 0.1, U columns 10 and V columns 15


In [10]:
results[0.1]

{'true_params': {'r': 10,
  'A': array([[-8.52834703e-04,  2.21479793e-03,  1.91137514e-04, ...,
           6.18556680e-05, -5.24768809e-04, -1.11917716e-03],
         [-2.41875944e-03,  2.03785050e-03, -1.01715786e-03, ...,
          -1.40815734e-03, -1.20692698e-03, -2.18639515e-03],
         [-1.19713602e-03, -1.58907714e-04,  2.43728498e-03, ...,
          -1.21372562e-05, -1.86304387e-03, -1.14379991e-03],
         ...,
         [-2.08890927e-03,  2.65792311e-03, -1.72470986e-03, ...,
           1.35300802e-03,  2.11621481e-03, -1.62374257e-03],
         [-3.00589681e-04, -4.31198827e-06, -3.30848006e-03, ...,
           2.12485587e-03,  2.39489905e-03, -3.36437543e-04],
         [-2.15123955e-03,  1.04177356e-03, -1.68503799e-03, ...,
           5.95059473e-04, -4.21443600e-05, -7.27423460e-04]]),
  'U_true': array([[-7.77390907e-03, -1.35430311e-02,  4.95309194e-03, ...,
          -3.34336788e-01, -2.09187985e-15,  3.66374116e-14],
         [ 2.30243138e-02, -8.73658131e-03,  6.